# Jupyter Setup

# Setup

In [2]:
# Jupyter Settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Import modules
import json
import math

# Import 
from simply_tiles.tms import TileMatrixSet

# Inhalt

*Im Folgenden wird die OGC Spezifikation [TileMatrixSet 2.0](https://docs.opengeospatial.org/DRAFTS/17-083r3.html) erläutert (derzeit im Draft Status). Zur umfassenden Illustration des TMS Konzepts wurden die wesentlichen Parameter und Berechnungen als eigene Python Klasse implementiert (vgl. `simply_tiles.tms.TileMatrixSet`). Die Implementierung nimmt den Vector-Tile Generator/Server T-Rex als Vorlage. Vergleiche dazu das entsprechende [T-Rex Modul im Quellcode](https://github.com/t-rex-tileserver/t-rex/blob/master/tile-grid/src/grid.rs)*

# Die 2D Tile Matrix

Eine einzelne 2D Tile Matrix (im folgenden auch zu TM verkürzt) ist ein Gitter über eine gegebene Bounding Box. Um sie begrifflich von der Bounding Box der Geometrien abzugrenzen, die auf Kacheln dargestellt werden sollen, wird sie fortan als „Extent“ bezeichnet. Laut TMS 2.0 wird das Gitter durch folgende Komponenten vollständig definiert:

* `extent` - *bounding box die in ein Kachelgitter unterteilt werden soll*
* `matrixWidht` und `matrixHeight` - *Zahl der Kacheln entlang X und Y, die den Extent vollständig bedecken sollen.*
* `tileSize` - *für X und Y in CRS Einheiten. Die Kachelgrößen folgen aus der zuvor gewählten Kachelzahl*
* `origin` - *Obere linke Ecke des Extents. Von hier aus werden die Kacheln entlang x (links nach rechts) und y (oben nach unten) gezählt als 0 basierter Index gezählt*

Weiterhin gibt es auch **tile matrix limits**, mit denen angegeben wird, in welchem Bereich der Tile Indizes Geometrien vorhanden sind. Die sind sowohl für Clients als auch Server nützlich. So muss ein Kachelgenerator z.B. nicht alle theoretisch möglichen Kacheln erzeugen sondern nur solche, in denen Geometrien zu sehen sind.

Interessant ist noch, dass sich die Zahl der Kacheln entlang X und Y erst im Nachhinein, also nach erfolgter Deinition einer Kachelmatrix ergeben. Anwender geben also keine arbiträre Anzahl an Kacheln vor. Stattdessen werden Gitter mittels räumlichen Auflösungsstufen und damit zusammenhängenden Maßstäben definiert. Wie das genau funktioniert, wird im weiteren Verlauf erläutert.

# Einzelne Kacheln der 2D Tile Matrix als Rasterkacheln rendern

Um ein Rasterkachel-Cache zu erzeugen, muss jede Kachel einzeln gerendert werden. Dafür wird sie in Pixel unterteilt. 
So ergibt sich ein weiteres „Gitter“ über jeder einzelnen Kachel, das als **extrapolated device grid** bezeichnet wird. Mit "device" ist ein „visualization device“ zum Rendern gemeint. Die Gitterräume werden als **grid cells** bezeichnet (fortan als Zellen übersetzt). Höhe und Breite einer (Pixel-)Zelle ergeben sich aus folgenden Vorgaben:

`tileWidth` und `tileHeight` - *Zahl der Pixel entlang X und Y. Meist identisch entlang X und Y, was die TMS Spec aber nicht vorschreibt*

`PixelScaleDenominator` - *Verhältnis der Pixelgröße zur realen Größe in Metern*

Die tatsächliche Pixelgröße eines Rendering Devices ist im Vorfeld nicht bekannt. Als Referenz nutzt die OGC daher einen quadratischen „Standardpixel“ von **0.28mm * 0.28mm** (wie schon in den WMS, SE und WMTS Spezifikationen). 

Was in der TMS Spec nicht direkt ersichtlich wird (zumindest für den Laien): Wozu braucht es den Standardpixel überhaupt? Würde das Rendering Device nicht ohnehin mit der tatsächlichen Pixelgröße rechnen? Des Rätsels Lösung: Der Standardpixel wird im Grunde als Näherungswert eingesetzt, um die (von Device zu Device variierende) Zellengröße einer gegebenen Tile Matrix in einen räumlichen Maßstab umzurechnen (und umgekehrt).

# Tile Matrix Sets und ihre Maßstäbe

Wie eingangs erwähnt, müssen sich Anwender zunächst überlegen, in welchen Maßstäben die Kacheln erzeugt werden sollen.
Pro Maßstab wird dann eine passende Tile Matrix gebildet. Es braucht also ein „Set“ an Maßsstäben, aus dem widerum ein "Set" an Kachelmatrizen folgt: das **Tile Matrix Set**.

* Jede Tile Matrix eines Sets bekommt laut TMS Spec einen „alphanumerischen Identifier“
* Das ist im Grunde äquivalent zum Konzept „Zoomstufe“, nur das sich die OGC hier allgemeiner fasst. Da in beiden Fällen Integer IDs vergeben werden, ist diese Differenzierung letztlich nebensächlich.

Übrigens: Innerhalb eines TMS kann die `tileWidth` entlang der Y-Achse variiert werden. Gängig ist z.B. der Ansatz, zu den Polen hin breitere Kacheln zu verwenden. Dafür wird das Tile Matrix Set um einen graduellen Parameter erweitert. Soweit ich die Spec richtig deute, wird das nur für Projektionen mit extremerer Verzerrung sowie in 3D Anwendungen genutzt. Für 2D Tiles in gängigen Projektionen sollte das nicht weiter relevant sein. Auch in [T-Rex](https://t-rex.tileserver.ch/) wurde der Parameter bisher nicht implementiert.

## Das TMS WebMercatorQuad und seine "krummen" Maßstäbe

Wenn man sich nun das gängige TMS [WebMercatorQuad](https://docs.opengeospatial.org/DRAFTS/17-083r3.html#web-mercator-quad-tilematrixset-definition-httpwww.opengis.netdeftilematrixsetogc1.0webmercatorquad) anschaut, stellt sich die Frage: Wie kommenn die krummen Maßstäbe zu Stande? 
Zur Illustration die Maßstäbe für Zoomstufen 0, 1 und 2: 

In [3]:
scale_denominator_level0 = 559082264.0287178
scale_denominator_level1 = 279541132.0143589
scale_denominator_level2 = 139770566.0071794

Warum sollte ein Anwender einen unintuitiven Maßsstab wie *1 : 559082264.0287178* wählen? Der Grund hierfür ist simpel: Im Falle von WebMercatorQuad folgen die Maßstäbe aus der "Kachellogik", nicht umgekehrt! Die Zahlen sind schlichtweg eine logische Folge aus der Entscheidung, den gesamten Extent der Projektion EPSG:3857 in eine **Bildpyramide** aufzuteilen  ([Illustration](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn.geotribu.fr%2Fimg%2Farticles-blog-rdp%2Fdivers%2FTilePyramid.jpg&f=1&nofb=1)). Im TMS 2.0 Spec ist das Konzept näher beschrieben. Im Grunde ist das eine besonders einfache und intuitive Weise ein Set an Kachelgittern zu definieren. Wäre die Bildpyramide eine Algorithmus, würde der Pseudocode wie folgt lauten:
* Nimm das gesamte CRS (in diesem Fall quadratisch) als Extent für alle Tile Matritzen im TMS
* Die erste Kachel umfasst den gesamten Extent und bekommt die Zoomstufe 0.
* Für jedes weitere Zoomlevel: teile Kachellänge und Kachelbreite durch 2 (= Verfierfachung der Kachelanzahl)
* Benutze zum Rendern 256x256 Pixel (`tileWidth` = `tileHeight`) pro Kachel 

So kommt es auch, dass sich die Maßstäbe mit jeder weiteren Zoomstufe halbieren:

In [4]:
(scale_denominator_level0 / 2) == scale_denominator_level1

True

## Zellgröße berechnen

Ausgehend vom geschilderten "Algorithmus" lassen sich nun die korrespondierenden, krummen Maßstäbe herleiten.
Zunächst brauchen wir aber die Zellengröße. Hier ein Beispiel für Zoomstufe 0:

In [23]:
# Extent des EPSG 3857 (in Metern)
extent = {
    "xmin" : -20037508.3427892,
    "ymin" : -20037508.3427892,
    "xmax" : 20037508.3427892,
    "ymax" : 20037508.3427892
}

# Länge bzw. Breite der quadratischen Kachel in CRS Einheiten bei Zoomstufe 0 (entspricht in diesem Fall dem Erdumfang)
tileSpan = extent["xmax"] - extent["xmin"] 

# Anzahl Pixel entlang X und Y einer jeden Kachel
tileHeight, tileWidth = 256, 256 

# Berechnung der Zellengröße
cellSize = tileSpan / 256 
cellSize 

156543.03392804062

Eine Pixellänge / Pixelbreite misst also ca. `156543` Meter bei Zoomstufe 0! Diese Größe wird auch als „Resolution Level“ bezeichnet. Präziser bleibt jedoch der Begriff „CellSize“ aus der TMS Spec. Schließlich ist hier nicht die Auflösung im gemeinen Sinne einer Bildschirm-Pixelzahl gemeint. Eher handelt es sich eine räumliche Auflösung, wie in diesen [Blogartikel](https://desktop.arcgis.com/de/arcmap/10.3/manage-data/raster-and-images/cell-size-of-raster-data.htm) beschrieben.

## Maßstab berechnen

Die Zellengröße ist im Grunde schon ein Maßstab: Meter pro Pixel!
Um den Maßstab auf eine beliebige Einheit (Meter, Fuß, Grad) zu generalisieren, muss die Referenzgröße eliminiert werden. Dafür brauchen wir die „physikalische“ Größe des Pixels im Bildschirm. Hier kommt der zuvor erwähnte "Standardpixel" ins Spiel:

In [6]:
standardPixelSize = 0.00028 # in Metern

cellSize / standardPixelSize # Einheitsloser Maßstab

559082264.0287166

Wenn das zu Grunde liegende CRS in Grad oder Fuß bemessen ist, so hätte eine cellSize die Einheit Grad/Pixel bzw. Fuß/Pixel. 
Hier muss die cellSize zunächst in Meter umgerechnet werden (erfolgt in dieser Implementierung automatisch)

## Unstimmigkeiten bei der Rundung

Am Rande: Der korrespondierende Wert zur Zoomstufe 0 aus der WebMercatorQuad Tabelle in der TMS Spec weicht minimal vom obigen Beispiel ab.
Teilt man jenen Wert durch die Standard Pixelgröße, entsteht wiederum ein minimal abweichender Wert vom tabellierten Maßstab.
Hier herrschen noch Unstimmigkeiten beim Handling von Rundungsfehlern vor. Auch bei T-Rex finden sich [rundungsbasierte Abweichungen](https://github.com/t-rex-tileserver/t-rex/blob/master/tile-grid/src/grid.rs).

In [24]:
cellSize_from_above_calculation = 156543.03392804062
cellSize_from_spec = 156543.0339280410
cellSize_from_spec / standardPixelSize 

559082264.0287179

## Zellgrößen und Maßstäbe abseits des Äquators

Wie schon erwähnt, dient die standardisierte Pixelgröße als Näherung. Es gibt aber noch einen weiteren Aspekt, der die Herleitung von Maßstäben verkompliziert: Die projektionsspezifische, geometrische Verzerrung. Bei der Mercator Projektion beispielsweise wird die Verzerrung zu den Polen größer. Folglich gibt es zur Bestimmung des Maßstabs eine [Korrektur](https://docs.microsoft.com/en-us/bingmaps/articles/understanding-scale-and-resolution), bei der der Breitengrad einbezogen wird:

`cellSize * cos(latitude)`

Die Zellengrößen und die daraus abgeleiteten Maßstäbe aus der WebMercatorQuad Definition gelten also **nur für den Äquator bzw. den Breitengrad 0**:

In [8]:
cellSize * math.cos(0)

156543.03392804062

In der Tabelle zum TMS „WorldCRS84Quad“ macht die TMS 2.0 Spec sogar explizit, dass die angegebenen Zellengrößen nur am Äquator gültig sind. Prinzipiell müsste sich dieser Hinweis in jeder TMS Tabelle wiederfinden. Zudem braucht jede Projektion vermutlich eine eigene Korrekturrechnung. Für das Erzeugen von Kacheln aus Vektorgeometrien ist das jedoch nebensächlich.

## Zusammenfassung

Startet man wie bei WebMercatorQuad mit einem "Kachel-Algorithmus", entsteht für jede Zoomstufe zunächst eine definierte  Kachelgröße bzw. `tileSpan`. Zusammen mit der Pixelzahl ergibt sich dann die Zellengröße und schließlich, unter Annahme eines "Standardpixels", der Maßsstab:

**Kachelgröße + Pixelzahl entlang X und Y --> Zellengröße <--> Maßstab**

Die formalisierte TMS Definition dreht diese Logik im Grunde nur um:

**Maßstab <--> Zellengröße --> Pixelzahl entlang X und Y + Kachelgröße**

Da sich Zellengröße und Maßstab jeweils voneinander ableiten lassen, genügt im Grunde nur einer von beiden Parametern.
Bei T-Rex wird ein Custom TMS beispielsweise nur mittels Zellgrößen (aka. „Resolutions“) definiert.
Bei GeoServer kann man zwischen beiden Größen wählen.

Weil die wahre Pixelgröße unbekannt ist, ist der Maßstab im TMS eine Näherung an den „wahren“ Maßstab, der sich aus dem Rendering Device ergibt. Der Rückgriff auf einen Standardpixel dient vermutlich dazu, eine Vergleichbarkeit zwischen unterschiedlichen TMS Definitionen herzustellen. Allerdings bedeutet es auch, dass Anwender mit Hilfe eines TMS keine exakten Maßstäbe vorgeben können!

# Kachelgitter aus TMS Parametern ableiten

Sollen Geometrien in Kacheln unterteilt werden, stellt sich weiterhin die Frage: Auf welchen Kacheln wären überhaupt Geometrien zu sehen? Ausgehend von der Bounding Box muss das für jede Zoomstufe einzeln beantwortet werden. Das TMS Spec schlägt dafür Pseudocode vor (siehe Annex I). Hier die simple Python Implementierung:

```
# Konstante um rundungsbasierte Abweichungen bei Dezimalzahlen
EPSILON = 0.0000001

# Auszug aus der Methode TileMatrixSet.tile_limits():
limits = {
    "tileMinCol": math.floor((bbox_xmin - tile_matrix_xmin) / tilespan_x + EPSILON),
    "tileMaxCol": math.floor((bbox_xmax - tile_matrix_xmin) / tilespan_x - EPSILON),
    "tileMinRow": math.floor((tile_matrix_ymax - bbox_ymax) / tilespan_y + EPSILON),
    "tileMaxRow": math.floor((tile_matrix_ymax - bbox_ymin) / tilespan_y - EPSILON)
}
```

## Beispiel: Vollständige Weltkarte in WebMercatorQuad

Als erstes laden wir alle relevanten TMS Parameter und instantiieren damit ein Objekt der Klasse `TileMatrixSet`. Diese ist mit allen wichtigen Formeln ausgestattet. Analog zu T-Rex werden die Zellengrößen (resolution level) zur TMS-Definition herangezogen. Die Herleitung von Maßsstäben (am Äquator) aus den Zellengrößen ist ebenfalls implementiert (zur Illustration).

In [25]:
# TMS Definition aus einer json Datei lesen
with open('data/WebMercatorQuad.json', mode="r") as json_data_file:
    tms_definition = json.load(json_data_file)

# TileMatrixSet Objekt basierend auf WebMercatorQuad instantiieren
tms = TileMatrixSet(**tms_definition)

# Die ersten 3 definierten Zellengrößen und die daraus abgeleiteten Maßstäbe:
for z in [0,1,2]:
    cell_size_z = tms.cell_sizes[z]
    scale_z = tms.scale_denominator(z)
    print(cell_size_z, scale_z)

156543.033928041 559082264.0287178
78271.5169640205 279541132.0143589
39135.75848201025 139770566.00717944


Zur Abbildung der gesamten Welt in WebMercatorQuad braucht es den gesamten EPSG:3857 `extent` als Bounding Box:

In [26]:
bbox = tms.extent
bbox

{'xmin': -20037508.342789248,
 'ymin': -20037508.342789248,
 'xmax': 20037508.342789248,
 'ymax': 20037508.342789248}

Sodann werden die Werte `tile_matrix_xmin` und `tile_matrix_ymax` benötigt. Hierbei handelt es sich um den `origin` der Kachel-Koordinaten. Üblicher Weise wird die obere linke Ecke vom Extent verwendet. Welche Ecke als Referenz gilt, ist prinzipiell beliebig. Bei T-Rex lässt sich z.B. auch "bottom left" in einer Custom TMS auswählen. Der Einfachheit halber wird in dieser Implementierung nur die obere linke Ecke angeboten. Wichtig ist nur, dass origin und die "Referenzecke" vom extent übereinstimmen. Aus diesem Grund muss in dieser Implementierung kein explizites Origin-Koordinatenpaar angegeben werden (analog zu T-Rex). Das TMS Objekt entnimmt die Werte aus dem Attribut `extent`. Im (skizzierten) XML und JSON Encoding der TMS Spec werden die Koordinaten hingegen explizit angegeben. 

In [28]:
tms.origin # statt expliziter Koordinaten wird in dieser Implementierung nur der String "top_left" zur Kenntlichmachung des Origins genutzt

tile_matrix_xmin, tile_matrix_ymax = tms.extent["xmin"], tms.extent["ymax"]
tile_matrix_xmin, tile_matrix_ymax

'top_left'

(-20037508.342789248, 20037508.342789248)

Bleiben noch Kachellänge und Kachelbreite bei gegebener Zoomstufe: `tilespan_x` und `tilespan_y`. Diese können aus der definierten Zellengröße und `tileWidth` bzw. `tileHeight` ermittelt werden:

In [29]:
z = 0 # Beispiel Zoomstufe zur Illustration

tilespan_x = tms.tile_width * tms.cell_sizes[z]
tilespan_y = tms.tile_height * tms.cell_sizes[z]
tilespan_x, tilespan_y #umspannen den kompletten Erdumfang entlang X und Y

(40075016.685578495, 40075016.685578495)

Jetzt sind alle Zutaten beisammen! Ausgehend von einer bounding box und einer Zoomstufe können nun die Kachelindizes ermittelt werden, innerhalb derer Geometrien vorhanden sind:

In [31]:
# Kachelindex Limits für diverse Zoomstufen (Illustration)
for z in [0, 4, 10]:
    tms.tile_limits(bbox, z) # Aus den Limits lassen sich auch tileWidth und tileHeight herleiten

{'tileMinCol': 0,
 'tileMaxCol': 0,
 'tileMinRow': 0,
 'tileMaxRow': 0,
 'matrixWidth': 1,
 'matrixHeight': 1}

{'tileMinCol': 0,
 'tileMaxCol': 15,
 'tileMinRow': 0,
 'tileMaxRow': 15,
 'matrixWidth': 16,
 'matrixHeight': 16}

{'tileMinCol': 0,
 'tileMaxCol': 1023,
 'tileMinRow': 0,
 'tileMaxRow': 1023,
 'matrixWidth': 1024,
 'matrixHeight': 1024}

## Beispiel: Beliebige Bounding Box in WebMercatorQuad

Meist werden Kachel-Caches nur für einen Teilbereich der gesamten Weltkarte erzeugt.
Solange unterschiedliche Caches ein und dasselbe TMS verwenden, lassen sie sich in GIS Clients besonders einfach übereinanderlegen.
Bestimmte Kacheln (z.B. die Kachel mit den Indizes bzw. "Kachekoordinaten" x=0, y=1, z=1) verweisen dann stets auf identische Koordinatenbereiche.

Wenn ein Client eine gegebene Bounding Box in ausgewählten Zoomstufen darstellt, würde er die anzufragenden Kacheln nach gezeigtem Prinzip ermitteln. Zur Illustration:

In [14]:
bbox = {
    "xmin": 50000,
    "ymin": 50000,
    "xmax": 100000,
    "ymax": 100000,  
} # in EPSG 3857

for z in [0, 4, 10]:
    tms.tile_limits(bbox, z)

{'tileMinCol': 0,
 'tileMaxCol': 0,
 'tileMinRow': 0,
 'tileMaxRow': 0,
 'matrixWidth': 1,
 'matrixHeight': 1}

{'tileMinCol': 8,
 'tileMaxCol': 8,
 'tileMinRow': 7,
 'tileMaxRow': 7,
 'matrixWidth': 1,
 'matrixHeight': 1}

{'tileMinCol': 513,
 'tileMaxCol': 514,
 'tileMinRow': 509,
 'tileMaxRow': 510,
 'matrixWidth': 2,
 'matrixHeight': 2}

## "Kachelkoordinaten" wieder in CRS Koordinaten umrechnen

Wie bisher dargelegt, lassen sich mittels TMS Parameter und einer gegebenen Bounding Box alle Kachelindizes bestimmen, auf denen Geometrien vorhanden sind.
Doch um entsprechende Kacheln zu erzeugen, müssen die Indizes wieder in eine Bounding Box umgerechnet werden. Nur so lassen sich Kacheln und die zu "kachelnden" Zielgeometrien zusammenbringen. Zur Abgrenzung vom TMS "Extent" wird die Bounding Box einer Kachel oft als `envelope` bezeichnet. Zur Ermittlung vom Envelope hat die TMS Spec ebenfalls Pseudocode parat. Hier wieder die Beispielimplementierung in Python:

```
# Auszug aus TileMatrixSet.tile_envelope():

envelope = {
            "xmin": tile_col * tilespan_x + tile_matrix_minx,
            "ymin": tile_matrix_maxy - (tile_row + 1) * tilespan_y,
            "xmax": (tile_col + 1) * tilespan_x + tile_matrix_minx,
            "ymax": tile_matrix_maxy - tile_row * tilespan_y
        }
```

Zur Illustration einmal der Envolpe zu einer der ingesamt 4 Kacheln, die sich aus der Bounding Box des vorausgehenden Beispiels bei Zoomstufe 10 in WebMercatorQuad ergeben:

In [15]:
z=10
x=513
y=509
tms.tile_envelope(x, y, z)

{'xmin': 39135.75848200917,
 'ymin': 78271.51696402207,
 'xmax': 78271.51696402207,
 'ymax': 117407.27544603124}

# Rechteckige Kacheln und Extents

Der TMS Pseudocode erlaubt zwar unterschiedliche Parameter für tileWidth und tileHeight, allerdings gibt es stets nur EINE einheitliche Zellengröße für X und Y.
Hier nochmal die Herleitung von Kachellänge (tile_span_y) und Kachelbreite (tile_span_x) in CRS Einheiten aus gegebenen TMS Parametern:

```
tileSpanX = tileWidth * cellSize
tileSpanY = tileHeight * cellSize
```

*Hinweis: Analog zur TMS Spec wird hier CamelCase benutzt, während die Python Implementierung snake_case benutzt*

Bei einer fixen Pixelgröße von 0.28mm und einer fixen Zellengröße lassen sich rechteckige Kacheln nur dann definieren, wenn `tileWidth` != `tileHeight` ausfällt.
In den TMS Beispielen der Spec sind tileWidth und tileHeight jedoch stets identisch (vgl. Abschnitt "Common TileMatrixSet definitions").
Aus der Spec geht auch nicht hervor, wann und warum beide Parameter voneinander abweichen sollten. Das erschließt sich wohl erst, wenn man tiefer in das Thema Rasterdaten einsteigt.

Übrigens spricht nichts dagegen, einen rechteckigen Extent in quadratische Kacheln zu unterteilen. Zu einer oder mehreren Seiten des Extents würden die Kacheln dann schlicht über den Extent hinausragen. Im T-Rex Beispiel für ein Custom TMS passiert genau das. Zur Illustration wurde das Beispiel in die  Python Implementierung aufgenommen:

In [16]:
with open('data/TrexCustomTMS.json', mode="r") as json_data_file:
    tms_definition = json.load(json_data_file)

custom_tms = TileMatrixSet(**tms_definition)

Hier einmal die Weite und Länge vom angegebenen Extent (in SRID=2056)

In [17]:
print("srid: ", custom_tms.srid)
print("units: ", custom_tms.units)

custom_extent = custom_tms.extent
extent_width = custom_extent["xmax"] - custom_extent["xmin"]
extent_height = custom_extent["ymax"] - custom_extent["ymin"]
extent_width, extent_height

srid:  2056
units:  meters


(480000.0, 320000.0)

Anhand der Zoomstufe 0 und bei gegebenen Pixelzahlen nun die Beispielberechnung:

In [32]:
# tms parameter zur Illustration
custom_tms.cell_sizes[0]
custom_tms.tile_width
custom_tms.tile_height

4000.0

256

256

Die Tile Limits lauten dann:

In [33]:
custom_tms.tile_limits(custom_extent, 0)

{'tileMinCol': 0,
 'tileMaxCol': 0,
 'tileMinRow': 0,
 'tileMaxRow': 0,
 'matrixWidth': 1,
 'matrixHeight': 1}

Und der Envelope zur resultierenden Kachel:

In [20]:
custom_envelope = custom_tms.tile_envelope(0,0,0)
custom_envelope

{'xmin': 2420000.0, 'ymin': 326000.0, 'xmax': 3444000.0, 'ymax': 1350000.0}

Wie erkennbar wird, entsteht hier eine quadratische Kachel, die deutlich über den Extent hinausragt:

In [36]:
envelope_width = custom_envelope["xmax"] - custom_envelope["xmin"]
envelope_height = custom_envelope["ymax"] - custom_envelope["ymin"]

print("Envelope Dimensions: ", envelope_width, envelope_height)
print("Extent Dimensions: ", extent_width, extent_height)

Envelope Dimensions:  1024000.0 1024000.0
Extent Dimensions:  480000.0 320000.0


# TMS bei Vector Tiles

Alle bis hierhin geschilderten Zusammenhänge gelten zunächst für Rasterkacheln.
Wie sich eine TMS Definition auf die Visualisierung von Vector Tiles auswirkt, lässt die TMS Spec weitgehend unkommentiert.
Verwirrend ist in diesem Kontext vor allem die Nutzung der Parameter tileWidth und tileHeight.
Denn anders als bei Rasterkacheln wird das komprimierter Binärformat (Protobuff), in dem Vector Tiles codiert sind, erst im Client gerendert! Nur so wird das nachträgliche Umstylen von Vector Tiles überhaupt möglich! Warum sollte man also im Vorfeld ein Pixelraster für jede einzelne Kachel festlegen? 

Ein aufschlussreicher Hinweis zum **extrapolated device grid** im Kontext von Vektordaten fällt in einer Randnotiz: *Some tiled vector data expressed in formats such as GeoJSON do not make use of an extrapolated device grid. Other tiled formats (e.g., MBTiles) define an internal coincident grid denser than the extrapolated device grid and express the position using indices in this denser grid instead of coordinates.*

Letzterer Satz bezieht sich zwar explizit auf MBTiles (abzugrenzen von Mapbox Vector Tiles). Allerdings trifft er auch auf die [Mapbox Vector Tiles Spezifikation](https://docs.mapbox.com/vector-tiles/specification/) zu. Auch hier werden geografische Koordinaten in ein "internal coincident grid" übertragen, was vor allem ihrer Komprimierung zu integer Werten dient. Im Grunde ist auch das eine Form von "Rasterisierung"! (*Bei genauerer Betrachtung sind alle Vektordaten auch Rasterdaten, da ein kontinuierliches Koordinatensystem immer diskret werden muss, sobald eine Koordinate mit einer fixen Zahl an Nachkommastellen festfehalten wird*)

Das Mapbox Grid ist in der Tat engmaschiger als das übliche Pixelraster, das bei den üblichen Parametern tileWidth = tileHeight = 256 (oder auch 512) Pixel entsteht. In der Spec fällt nämlich der Wert **4096**. Die Rede ist von einem "extent", gemeint sind jedoch 4096 * 4096 Zellen. Also das "lokale" Koordinatensystem einer einzelnen Vektorkachel (zumindest nach meinem Verständnis). Da der Wert in der Spezifikation nur als Beispiel herangezogen wird, bleibt seine Herleitung unklar. Die [PostGIS Implementierung der MVT Spezifikation](https://postgis.net/docs/ST_AsMVTGeom.html) hat ihn als Default übernommen, leider ebenfalls ohne nähere Erläuterung. Womöglich ist der Wert bereits so engmaschig, dass man sich in der Praxis nicht mehr um die räumliche Auflösung sorgen muss.

Was jedoch klar wird: Die Zellengrößen, die ein TMS vorgibt, sind **UNABHÄNGIG** vom *internal coincident grid* einer einzelnen Vektorkachel.
Das TMS beeinflusst Vector Tiles nur insofern, als dass es die Größe der Kacheln (bzw. ihren Envelope) steuert:
* Je kleiner die Zellengröße, desto mehr Kacheln fallen bei gegebener Zoomstufe an.
* Je mehr Kacheln anfallen, desto kürzer ihre Seitenlängen: die BBOX der Kachel bzw. der Tile Envelope schrumpft!
* Je kleiner die Kacheln, desto feiner die räumliche Auflösung des "extrapolated device grids". Nur, dass sich jede Kachel nicht mehr in tileWith * tileHeight unterteilt, sondern in das engmaschigere Gitter von 4096 * 4096 Zellen (sofern man den Default beibehält).

**Somit haben Vector Tiles stets eine größere räumliche Auflösung (bzw. kleinere Zellengrößen) als es das TMS suggeriert**

## Fazit

Auch wenn das TMS Konzept im Hinblick auf Rasterdaten entstandt und in Anwendung auf Vector Tiles verwirrend sein kann, bleibt es eine essenzielle Standardisierung zur Erzeugung und Darstellung gekachelter Geodaten. Nur bei bekanntem TMS kann der Client die richtigen Kacheln anfragen und an der richtigen Stelle rendern. Dazu muss er die hier illustrierten Methoden zur Bestimmung von Tile Limits und des Tile Envelope implementieren. Dasselbe gilt für Software, die basierend auf einem TMS Tile-Caches erzeugt.

Nicht alle Clients sind in dieser Hinsicht konsequent bzw. explizit. So setzt Mapbox ausschließlich auf WebMercatorQuad, setzt also einen impliziten Standard. Besondere Vorsicht ist auch bei QGis geboten: Die Berechnungen gehen hier pauschal vom TMS `GoogleCRS84Quad` aus.
In der allgemeinen Doku wird das leider nicht erwähnt, immerhin jedoch in der Dokumentation zur [QGIS API](https://qgis.org/api/3.20/classQgsVectorTileUtils.html). Lädt man dennoch einen Tile-Cache in WebMercatorQuad, werden die Kacheln trotzdem an richtiger Stelle positioniert.
Das funktioniert nur deshalb, weil beide TMS auf dasselbe "Bildpyramidenprinzip" zurückgreifen. Trotz unterschiedlicher Projektionen resultieren in diesem exaten Fall identische Maßstäbe und somit auch identische Kachedimensionen. Folglich zeigt eine Kachel mit den exemplarischen Indizes (x=0, y=1, z=1) sowohl in WebMercatorQuad als auch in GoogleCRS84Quad ein und denselben geografischen Ausschnitt!